<a href="https://colab.research.google.com/github/vixbi/actprobl/blob/main/Project_OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Создание рабочей папки для работы с Kaldi**



## **Критерии:**
1. Использование классов. (2 балла)
2. Использование наследования. (2 балла)
3. Три библиотеки из тех, что приходится ставить своими руками. (1 балла)
4. "Синтаксический сахар" Питона (декораторы, перегрузка операторов, исключения, примочки из последнего стандарта, ...). (2 балла)
5. Юнит-тесты. (1 балл)
6. Красивая структура классов, проекта, решения; субъективное мнение преподавателя (2 балла).
7. Шаблоны проектирования (1 балл).**Текст, выделенный полужирным шрифтом**

## **Цели и результат**
Существует инструмент Kaldi, позволяющий обучать модели для автоматического распознавания речи на собственном материале. К сожалению, у Kaldi отсутствует интерфейс, а ручная подготовка материала к работе на собственном материале занимает достаточнот много времени и требует огромной внимательности даже при следовании самому базовому туториалу.

Данный проект призван облегчить работу пользователя с Kaldi в части подготовки обучающего и тестового материалов.

*   На входе принимаются директории, в которых находятся необходимые аудио- и видео- данные, а также директория, в которую необходимо поместить результат (дефолт - ...)
*   На выходе пользователь получает готовую для работы с Kaldi папку.


## **Планируемая структура кода**
- input пользователя: директория аудио- и текстового файла; try-except в случае отсутствия файлов в указанных директориях, желаемая работа с Kaldi (хочет ли он работать со стандартными транскрипциями и классическим словарем или со словарем аллофонов)
- класс *File*: абстрактный класс
- классы *Textgrid* и *Simplified_TextGrid* (наследуют File): преобразуют нетипичные форматы в нормальные форматы для работы Utterance
- класс *Utterance*: создает объекты типа Utterance, о которых определена следующая информация (возможно, переопределить тут __add__):
 - ID высказывания
 - тайм-код высказывания
 - орфографическая расшифровка
 - транскрипция высказывания
 - файл, которому принадлежит высказывание
- Юнит-тесты:
 - проверяют, в нужном ли формате файлы (для класса File)
 - проверяют, совпадает ли предлагаемый текстовый файл с аудиофайлом (если у них разные тайм-коды, то как бы да)

Еще штуки:
- декоратор @progress_bar для показа прогресса по разным процессам

---

 **Библиотеки:**
```
import re # работа с текстом транскрипции (его предобработка)
import os # работа с директориями
from sklearn.model_selection import train_test_split # деление на выборки
from pydub import AudioSegment # нарезка
from progress.bar import IncrementalBar # отслеживает процесс нарезки
```


In [ ]:
import os
import re
from sklearn.model_selection import train_test_split

In [ ]:
# функция для обработки файлов STG
def SimplifiedTextGreed(file):
  with open(file) as f:
    masha = [[i.strip() for i in (line.split("\t"))] for line in f]

  def contains_cyrillic(text):
    return bool(re.search(r'[\u0400-\u04FF]', text))

  future_list = []
  for i in masha:
    t_list = []
    if contains_cyrillic(i[2]) == True and i[2] != "<нрзб>":
        t_list.append(i[2])
        t_list.append('[imitation]')
        t_list.append(f"{i[0]} {i[3]}")
        future_list.append(' '.join(t_list))

  return future_list

In [ ]:
with open('guest01.txt') as file:
    gue01 = [[i.strip() for i in re.split('\[|\]', line)]
             for line in file if '[' in line and 'хезит' not in line]

In [ ]:
class Utterance:
    def __init__(self, list_for_line, k):
      self.orpho = re.sub(r'\W', r'_', list_for_line[0])
      self.trans = re.sub(r'\d\?\)\(', r'', list_for_line[1])
      self.timecode = [float(time)*1000 for time in list_for_line[2].split(' ')]
      self.id = k

    def __repr__(self):
        return f"{self.orpho, self.trans, self.timecode, self.id}"

In [ ]:
class File:
  def __init__(self):


In [ ]:
a = Utterance(gue01[0], 0)
a

('утренний_гость', "utr'in'i goIs't'", [5301.0, 6740.0], 0)

# **Коробочка. Упражнение**

In [1]:
class BoxException(Exception):
  def __init__(self, message):
    self.message = message
    super().__init__(message)

  def __str__(self):
    return f"{self.message}"

In [2]:
def b(k, g):
  if k/g > 1:
    raise BoxException("Oh, I guess there's no space left")
  return k/g


In [20]:
def well_decorator(func):
  def new_func(*args, **kwargs):
    try:
      func(*args, **kwargs)
      return True
    except:
      return False
  return new_func

@well_decorator
def names(a, b):
  print(a, b)

In [21]:
names('lada', 'anton')

lada anton


True

In [61]:
class Korobochka:
  def __init__(self, a, b, c):
    self.shirina = a
    self.dlina = b
    self.vysota = c
    self.__list = []
    self.__fulled = 0

  def objem(self):
    return self.shirina*self.dlina*self.vysota

  def enlarge(self, k):
    self.shirina *= k
    self.dlina *= k
    self.vysota *= k

  def l(self):
    if self.objem() < self.__fulled:
      raise BoxException("Oh, I guess there's no space left")
    return self.objem()/self.__fulled

  def put(self, new_korobochka):
    if self.objem() < (self.__fulled + new_korobochka.objem()):
      raise BoxException("Oh, I guess there's no space left")
    else:
      self.__list.append(new_korobochka)
      self.__fulled += new_korobochka.objem()

  def get_items(self):
    return self.__list


  def __repr__(self):
    return f"""Hello, I am a Box! My length is {self.dlina},
    my width is {self.shirina}, my height is {self.vysota}.
    I am not that large, my volume is only {self.objem()}. You \
    can enlarge me, using Box.enlarge()"""

In [62]:
import unittest

In [63]:
class TestCorobochka(unittest.TestCase):
  def setUp(self):
    self.k1 = Korobochka(4, 6, 8).objem()
    self.k2 = Korobochka(1, 1, 1).put(Korobochka(1, 4, 1))

  def test_objem(self):
     self.assertEqual(self.k1, 192)

  def test_put(self):
     self.assertEqual(self.k2, False)

In [64]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_objem (__main__.TestCorobochka) ... ERROR
test_put (__main__.TestCorobochka) ... ERROR

ERROR: test_objem (__main__.TestCorobochka)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-63-cb1122acec2b>", line 4, in setUp
    self.k2 = Korobochka(1, 1, 1).put(Korobochka(1, 4, 1))
  File "<ipython-input-61-204bd04d57ae>", line 24, in put
    raise BoxException("Oh, I guess there's no space left")
BoxException: Oh, I guess there's no space left

ERROR: test_put (__main__.TestCorobochka)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-63-cb1122acec2b>", line 4, in setUp
    self.k2 = Korobochka(1, 1, 1).put(Korobochka(1, 4, 1))
  File "<ipython-input-61-204bd04d57ae>", line 24, in put
    raise BoxException("Oh, I guess there's no space left")
BoxException: Oh, I guess there's no space left

---------------------------------------

In [26]:
a = Korobochka(30, 40, 50)
a.put(Korobochka(3, 4, 5))

True

In [ ]:
class SquaredBox(Korobochka):
  def __init__(self, a):
    Korobochka.__init__(self, a, a, a)


In [ ]:
class Parcel(Korobochka):
  def __init__(self, a, b, c, w, dir):
    Korobochka.__init__(self, a, b, c)
    self.__w = w
    self.__dir = dir

  def get_weight(self):
    return self.__w

  def set_weight(self, w):
    self.__w = w

  def destination(self):
    return self.__dir

In [ ]:
d = Parcel(1, 2, 3, 50, 'Jopa-Zajopa')
print(d.get_weight())
d.set_weight(100)
print(d.get_weight())

50
100


In [ ]:
c = SquaredBox(4)
print(c.objem())

64


In [ ]:
from copy import deepcopy

a = Korobochka(1, 2, 3)
b = deepcopy(a)
a.enlarge(5)
print(a.objem()/b.objem())

125.0
